In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver

In [2]:
# 通过url获取response 仅使用resquests
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    return response

In [3]:
# 获取页面的html文件
def getHTML(url):
    return getResponse(url).text

In [4]:
# <audio src="https://file.bohe.cn/fhfile1/M00/02/0B/oYYBAGA-6OGAJJS8AA3GaAzTvw8685.mp3"></audio>

In [5]:
# 通过音频源地址爬取音频
def getAudio(url):
    response = getResponse(url)
    path = 'audio/'
    fileName = url.split('/')[-1]
    with open(path + fileName, 'wb') as fp:
        fp.write(response.content)

In [6]:
getAudio('https://file.bohe.cn/fhfile1/M00/02/0B/oYYBAGA-6OGAJJS8AA3GaAzTvw8685.mp3')

In [7]:
url = 'https://www.bohe.cn/voice/view/8rbz6mi8k258ukp.html'
html = getHTML(url)
print(html)

<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="renderer" content="webkit">
    <meta http-equiv="Cache-Control" content="no-transform"/> 
    <meta http-equiv="Cache-Control" content="no-siteapp"/>
    <meta name="applicable-device" content="pc">
    <meta name="mobile-agent" content="format=xhtml;url=https://m.bohe.cn/voice/view/8rbz6mi8k258ukp.html">
    <link type="image/x-icon" href="//static.bohe.cn/favicon.ico" rel="shortcut icon">
    <link rel="alternate" media="only screen and(max-width: 640px)" id="toM" href="https://m.bohe.cn/voice/view/8rbz6mi8k258ukp.html">
    <meta name="keywords" content="肺结核,肺结核饮食禁忌,肺结核病人不能吃什么,语音科普">
    <meta name="description" content="肺结核病人首应避免食用辛辣、刺激、油腻等难以消化的食物，这样可加重胃肠负担，不利于消化吸收；还可刺激引...">
    <title>肺结核病人不能吃什么 - 语音科普 - 博禾医生</title>
    <link rel="stylesheet" href="https://static.bohe.cn/video/pc/home/css/base.inc.css?v=23">
    <link rel="stylesheet" href="https://

In [8]:
"""
<div class="dis-txt-info">肺结核病人首应避免食用辛辣、刺激、油腻等难以消化的食物，这样可加重胃肠负担，不利于消化吸收；还可刺激引起咳嗽症状加重；还有避免吃热性、易上火的食物，如荔枝、龙眼；还需避免吃寒凉食物，可能会导致腹泻，使免疫功能减低，不利于病情恢复；还应避免食用海鲜类，如螃蟹、海产品等，这类食物属于发物，有加重炎症反应的可能。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;同时还应避免饮酒，肺结核患者需要行抗结核治疗，饮酒可能会加重肝脏负担，影响药物疗效；避免吸烟，减少呼吸道刺激。<audio src="https://file.bohe.cn/fhfile1/M00/02/0B/oYYBAGA-6OGAJJS8AA3GaAzTvw8685.mp3"></audio></div>
"""

'\n<div class="dis-txt-info">肺结核病人首应避免食用辛辣、刺激、油腻等难以消化的食物，这样可加重胃肠负担，不利于消化吸收；还可刺激引起咳嗽症状加重；还有避免吃热性、易上火的食物，如荔枝、龙眼；还需避免吃寒凉食物，可能会导致腹泻，使免疫功能减低，不利于病情恢复；还应避免食用海鲜类，如螃蟹、海产品等，这类食物属于发物，有加重炎症反应的可能。<br>\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;同时还应避免饮酒，肺结核患者需要行抗结核治疗，饮酒可能会加重肝脏负担，影响药物疗效；避免吸烟，减少呼吸道刺激。<audio src="https://file.bohe.cn/fhfile1/M00/02/0B/oYYBAGA-6OGAJJS8AA3GaAzTvw8685.mp3"></audio></div>\n'

In [9]:
# 获取语音对应的文本
def getAudioText(html):    
    ex = '<div class="dis-txt-info">(.*?)</div>'
    audio_text = re.findall(ex, html, re.S)
    return audio_text[0]

In [10]:
print(getAudioText(html))

肺结核病人首应避免食用辛辣、刺激、油腻等难以消化的食物，这样可加重胃肠负担，不利于消化吸收；还可刺激引起咳嗽症状加重；还有避免吃热性、易上火的食物，如荔枝、龙眼；还需避免吃寒凉食物，可能会导致腹泻，使免疫功能减低，不利于病情恢复；还应避免食用海鲜类，如螃蟹、海产品等，这类食物属于发物，有加重炎症反应的可能。<br />
       同时还应避免饮酒，肺结核患者需要行抗结核治疗，饮酒可能会加重肝脏负担，影响药物疗效；避免吸烟，减少呼吸道刺激。


In [11]:
# <audio src="https://file.bohe.cn/fhfile1/M00/02/0B/oYYBAGA-6OGAJJS8AA3GaAzTvw8685.mp3"></audio>

In [12]:
# 根据音频播放页面地址获取音频源地址
def getAudioSrcUrl(html):
    ex = '<audio src="(.*?)"></audio>'
    audio_src_url = re.findall(ex, html, re.S)
    return audio_src_url[0]

In [13]:
print(getAudioSrcUrl(html))

https://file.bohe.cn/fhfile1/M00/02/0B/oYYBAGA-6OGAJJS8AA3GaAzTvw8685.mp3


In [14]:
# <a href="https://www.bohe.cn/voice/view/8rbz6mi8k258ukp.html">

In [15]:
# 从首页获取各个播放页地址
url = 'https://www.bohe.cn/voice/list/new/0_0_1.html'
html = getHTML(url)
print(html)

<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="renderer" content="webkit">
    <meta http-equiv="Cache-Control" content="no-transform"/> 
    <meta http-equiv="Cache-Control" content="no-siteapp"/>
    <meta name="applicable-device" content="pc">
    <meta name="mobile-agent" content="format=xhtml;url=https://m.bohe.cn/voice/list/1.html">
    <link type="image/x-icon" href="//static.bohe.cn/favicon.ico" rel="shortcut icon">
    <link rel="alternate" media="only screen and(max-width: 640px)" id="toM" href="https://m.bohe.cn/voice/list/1.html">
    <meta name="keywords" content="全部,语音科普">
    <meta name="description" content="博禾医生的语音频道是中国专业的健康音频网站，包括各种健康音频讲座、名医访谈、健康科普音频等。更多权威健康知识，尽在博禾医生语音频道。
">
    <title>全部 - 语音科普 - 博禾医生</title>
    <link rel="stylesheet" href="https://static.bohe.cn/video/pc/home/css/base.inc.css?v=23">
    <link rel="stylesheet" href="https://static.bohe.cn/video/pc/voice/css/base.inc.

In [16]:
# 获取每一页的音频播放页面链接
def getAllLinkOfEveryPage(url):
    html = getHTML(url)
    soup = BeautifulSoup(html, 'html.parser')
    all_a_href = soup.find_all('a')

    linkLists = []
    for href in all_a_href:
        temp = str(href)
        matchObj = re.match(r'<a href="https://www.bohe.cn/voice/view/.*html', temp, re.M|re.I)
        if matchObj:
            ex = '<a href="(.*?).html'
            temp = re.findall(ex, temp, re.S)
            linkLists.append(temp[0] + '.html')
    linkLists = list(set(linkLists))
    return linkLists

In [17]:
for link in getAllLinkOfEveryPage('https://www.bohe.cn/voice/list/new/0_0_1.html'):
    print(link)

https://www.bohe.cn/voice/view/d09spxgkk51gtho.html
https://www.bohe.cn/voice/view/n1pvcd3bkrn0qu3.html
https://www.bohe.cn/voice/view/rsr8cbl5km65p9x.html
https://www.bohe.cn/voice/view/79wc608nkxi3d1d.html
https://www.bohe.cn/voice/view/jz65kws9k2nc6bt.html
https://www.bohe.cn/voice/view/z5lqk8yjknrkqpy.html
https://www.bohe.cn/voice/view/7hpskrumk2wgnsk.html
https://www.bohe.cn/voice/view/xcel6leck6uojsw.html
https://www.bohe.cn/voice/view/i0b4kk1wkoih6oc.html
https://www.bohe.cn/voice/view/0h7oknjrkthdb61.html
https://www.bohe.cn/voice/view/w9d413hlkomiqjv.html
https://www.bohe.cn/voice/view/punu6o9xknu47nw.html
https://www.bohe.cn/voice/view/qaz2uqm0ktm1a2f.html
https://www.bohe.cn/voice/view/xtzvccpzkmphyml.html
https://www.bohe.cn/voice/view/8rbz6mi8k258ukp.html
https://www.bohe.cn/voice/view/06g46twok5pll0f.html
https://www.bohe.cn/voice/view/0mnxrj4uk37jxuu.html
https://www.bohe.cn/voice/view/0gnyr5c2kmfqbyh.html
https://www.bohe.cn/voice/view/ljpef6ktktmv9g0.html
https://www.

In [18]:
# https://www.bohe.cn/voice/list/new/0_0_2.html
# 根据相关url爬取数据
def getTextAndAudio(url):
    html = getHTML(url)
    srcUrl = getAudioSrcUrl(html)
    text = getAudioText(html)
    getAudio(srcUrl)
    fileName = srcUrl.split('/')[-1]
    fileName = fileName.replace("mp3", "txt")
    print(fileName)
    # 写入文本数据
    with open('text/' + fileName, 'w', encoding='utf-8') as fp:
        fp.write(text)
    print(fileName, "successful")

In [19]:
for i in range(1, 101):
    pageUrl = 'https://www.bohe.cn/voice/list/new/0_0_' + str(i) + '.html'
    linkLists = getAllLinkOfEveryPage(pageUrl)
    for link in linkLists:
        getTextAndAudio(link)
        print("a audio successful")
    print("a page successful")

oYYBAGA-6OGAF2XyAA65aHBLmXw098.txt
oYYBAGA-6OGAF2XyAA65aHBLmXw098.txt successful
a audio successful
o4YBAGA-6MqAWCu7AA2LcJNtwWY004.txt
o4YBAGA-6MqAWCu7AA2LcJNtwWY004.txt successful
a audio successful
ooYBAGA-6MqAbFZTAA2rgFQi_AU269.txt
ooYBAGA-6MqAbFZTAA2rgFQi_AU269.txt successful
a audio successful
oYYBAGA-6MqAAb4iAA1KGPBk9Nw524.txt
oYYBAGA-6MqAAb4iAA1KGPBk9Nw524.txt successful
a audio successful
o4YBAGA-6OKAW5NRAA4bcGs2gWE551.txt
o4YBAGA-6OKAW5NRAA4bcGs2gWE551.txt successful
a audio successful
ooYBAGA-6OKAHGLoAA2_MH5sTyM931.txt
ooYBAGA-6OKAHGLoAA2_MH5sTyM931.txt successful
a audio successful
o4YBAGA-6OGAb8qbAA6lEC7kCQA561.txt
o4YBAGA-6OGAb8qbAA6lEC7kCQA561.txt successful
a audio successful
ooYBAGA-6OKAIwMIABEsmCrkE84998.txt
ooYBAGA-6OKAIwMIABEsmCrkE84998.txt successful
a audio successful
ooYBAGA-6OGAU_aEAA5UuMHkVBE247.txt
ooYBAGA-6OGAU_aEAA5UuMHkVBE247.txt successful
a audio successful
ooYBAGA-6MqAQW1EAAy0qNpoXbs035.txt
ooYBAGA-6MqAQW1EAAy0qNpoXbs035.txt successful
a audio successful


o4YBAGA-58KACSVMABHygMnHve8918.txt
o4YBAGA-58KACSVMABHygMnHve8918.txt successful
a audio successful
ooYBAGA-596AAln-AA6w-GpPL20787.txt
ooYBAGA-596AAln-AA6w-GpPL20787.txt successful
a audio successful
o4YBAGA-596AI9YuAA_4EA5UsHU119.txt
o4YBAGA-596AI9YuAA_4EA5UsHU119.txt successful
a audio successful
oYYBAGA-58KAYSd_ABIsoFgE6Os779.txt
oYYBAGA-58KAYSd_ABIsoFgE6Os779.txt successful
a audio successful
ooYBAGA-58KARs-wAA8hGMLA9rs234.txt
ooYBAGA-58KARs-wAA8hGMLA9rs234.txt successful
a audio successful
ooYBAGA-58KAJwbtAA4CUCqIvnw027.txt
ooYBAGA-58KAJwbtAA4CUCqIvnw027.txt successful
a audio successful
o4YBAGA-596AdfDrAA8tkDP4zKg868.txt
o4YBAGA-596AdfDrAA8tkDP4zKg868.txt successful
a audio successful
o4YBAGA-596AREoCAA_hYGnFrNE085.txt
o4YBAGA-596AREoCAA_hYGnFrNE085.txt successful
a audio successful
o4YBAGA-58KALqqRAA7TkIwmmAg091.txt
o4YBAGA-58KALqqRAA7TkIwmmAg091.txt successful
a audio successful
ooYBAGA-58KAX6hKABQrkH2kemc580.txt
ooYBAGA-58KAX6hKABQrkH2kemc580.txt successful
a audio successful


o4YBAGBAn0uAMGo3AA2s8gxfwCE625.txt
o4YBAGBAn0uAMGo3AA2s8gxfwCE625.txt successful
a audio successful
o4YBAGBAn0uASBToAAQ-0uLC8Ic338.txt
o4YBAGBAn0uASBToAAQ-0uLC8Ic338.txt successful
a audio successful
oYYBAGBAn2SAbx0UAAfePBeGvBw304.txt
oYYBAGBAn2SAbx0UAAfePBeGvBw304.txt successful
a audio successful
oYYBAGBAn2SABERAAAPTeR1sHAA557.txt
oYYBAGBAn2SABERAAAPTeR1sHAA557.txt successful
a audio successful
o4YBAGBAn2SAKT_wAAShMh9rD1A574.txt
o4YBAGBAn2SAKT_wAAShMh9rD1A574.txt successful
a audio successful
o4YBAGBAn2SAJR3gAAoJhNc3Rp0997.txt
o4YBAGBAn2SAJR3gAAoJhNc3Rp0997.txt successful
a audio successful
ooYBAGBAn0uAG-QBAAQ34jkoaYY180.txt
ooYBAGBAn0uAG-QBAAQ34jkoaYY180.txt successful
a audio successful
ooYBAGBAn2SAJbTIAAodTcPUGmg365.txt
ooYBAGBAn2SAJbTIAAodTcPUGmg365.txt successful
a audio successful
ooYBAGBAn2SANaNPAAR6zvKThSg848.txt
ooYBAGBAn2SANaNPAAR6zvKThSg848.txt successful
a audio successful
o4YBAGBAn2SAZIZmAAR0RkvMaE0346.txt
o4YBAGBAn2SAZIZmAAR0RkvMaE0346.txt successful
a audio successful


o4YBAGBAneiAOar4AAlRvSQuakQ954.txt
o4YBAGBAneiAOar4AAlRvSQuakQ954.txt successful
a audio successful
oYYBAGBAneiAQsg8AAiuaLq57pI622.txt
oYYBAGBAneiAQsg8AAiuaLq57pI622.txt successful
a audio successful
ooYBAGBAneiAXqDsAAgc7jTovlg704.txt
ooYBAGBAneiAXqDsAAgc7jTovlg704.txt successful
a audio successful
ooYBAGBAneiAEJfTAAlrrtfOtpk630.txt
ooYBAGBAneiAEJfTAAlrrtfOtpk630.txt successful
a audio successful
o4YBAGBAneiAMS0RAAj5I6RT6w4454.txt
o4YBAGBAneiAMS0RAAj5I6RT6w4454.txt successful
a audio successful
o4YBAGBAneiAVkJWAAjx2JTOoSU511.txt
o4YBAGBAneiAVkJWAAjx2JTOoSU511.txt successful
a audio successful
oYYBAGBAnc6AeFSZAAkDsyJewHM826.txt
oYYBAGBAnc6AeFSZAAkDsyJewHM826.txt successful
a audio successful
o4YBAGBAneiASb9WAAiiLgcSULc969.txt
o4YBAGBAneiASb9WAAiiLgcSULc969.txt successful
a audio successful
ooYBAGBAneiALnIOAAfYuPSAlNo151.txt
ooYBAGBAneiALnIOAAfYuPSAlNo151.txt successful
a audio successful
ooYBAGBAnc6AKyX6AAkDwscExk8857.txt
ooYBAGBAnc6AKyX6AAkDwscExk8857.txt successful
a audio successful


ooYBAGBAmh6AXzaAAAgHBbSM3mo914.txt
ooYBAGBAmh6AXzaAAAgHBbSM3mo914.txt successful
a audio successful
ooYBAGBAmh6AJ6ttAAlC9t3mpPQ560.txt
ooYBAGBAmh6AJ6ttAAlC9t3mpPQ560.txt successful
a audio successful
ooYBAGBAmh6AMT8UAAe2nXfV9lc713.txt
ooYBAGBAmh6AMT8UAAe2nXfV9lc713.txt successful
a audio successful
ooYBAGBAmh6AP-6dAAlSd035E0Q950.txt
ooYBAGBAmh6AP-6dAAlSd035E0Q950.txt successful
a audio successful
o4YBAGBAmh6AX3m6AAjYAhWLJyk695.txt
o4YBAGBAmh6AX3m6AAjYAhWLJyk695.txt successful
a audio successful
ooYBAGBAmh6AF8oRAAkSxmNq5bY311.txt
ooYBAGBAmh6AF8oRAAkSxmNq5bY311.txt successful
a audio successful
o4YBAGBAmqSARFSsAAozxbH_mb0665.txt
o4YBAGBAmqSARFSsAAozxbH_mb0665.txt successful
a audio successful
o4YBAGBAmh6AMF4GAAPuZQnGNU0045.txt
o4YBAGBAmh6AMF4GAAPuZQnGNU0045.txt successful
a audio successful
oYYBAGBAmqeAEuDsAAg1lXQNSsA261.txt
oYYBAGBAmqeAEuDsAAg1lXQNSsA261.txt successful
a audio successful
oYYBAGBAmqSAAeGBAAgeuFMcEcg979.txt
oYYBAGBAmqSAAeGBAAgeuFMcEcg979.txt successful
a audio successful


...

o4YBAGAPuG6AY4Q7ABiXnA9WoRQ262.txt
o4YBAGAPuG6AY4Q7ABiXnA9WoRQ262.txt successful
a audio successful
o4YBAGAPuG6AO1UCABhxXGHsVUI858.txt
o4YBAGAPuG6AO1UCABhxXGHsVUI858.txt successful
a audio successful
o4YBAGAPuEmAFZjTABg9nJBJxic531.txt
o4YBAGAPuEmAFZjTABg9nJBJxic531.txt successful
a audio successful
ooYBAGAPuEmAXcacABoB3F8b4LQ218.txt
ooYBAGAPuEmAXcacABoB3F8b4LQ218.txt successful
a audio successful
o4YBAGAPuEmAJGpIABw0XPZRKMc285.txt
o4YBAGAPuEmAJGpIABw0XPZRKMc285.txt successful
a audio successful
oYYBAGAPuG6Af3FnABcJXEGBYCI039.txt
oYYBAGAPuG6Af3FnABcJXEGBYCI039.txt successful
a audio successful
oYYBAGAPuG6AVk44ABfT3OywsuM929.txt
oYYBAGAPuG6AVk44ABfT3OywsuM929.txt successful
a audio successful
o4YBAGAPuG6AdwdCABoB3GgZjXE012.txt
o4YBAGAPuG6AdwdCABoB3GgZjXE012.txt successful
a audio successful
o4YBAGAPuEmAVOfoABqLHErKZ68464.txt
o4YBAGAPuEmAVOfoABqLHErKZ68464.txt successful
a audio successful
ooYBAGAPuG-AQhhKABnpHNcoAzo490.txt
ooYBAGAPuG-AQhhKABnpHNcoAzo490.txt successful
a audio successful


ooYBAGAPtsuAdspnABk72zKb_lc954.txt
ooYBAGAPtsuAdspnABk72zKb_lc954.txt successful
a audio successful
o4YBAGAPto6AFQjVABf6HLvTcHk833.txt
o4YBAGAPto6AFQjVABf6HLvTcHk833.txt successful
a audio successful
o4YBAGAPtrCAbSqkABZwXKHDiWg254.txt
o4YBAGAPtrCAbSqkABZwXKHDiWg254.txt successful
a audio successful
o4YBAGAPtsuAc5MhABhdG5KsCes523.txt
o4YBAGAPtsuAc5MhABhdG5KsCes523.txt successful
a audio successful
ooYBAGAPtrCAGlzMABed3IqDOkA535.txt
ooYBAGAPtrCAGlzMABed3IqDOkA535.txt successful
a audio successful
ooYBAGAPtsuACHL5ABe42znoq5Y219.txt
ooYBAGAPtsuACHL5ABe42znoq5Y219.txt successful
a audio successful
oYYBAGAPtsuAWUGXABmD29icItM010.txt
oYYBAGAPtsuAWUGXABmD29icItM010.txt successful
a audio successful
o4YBAGAPtsyAeRITABlCm7lv5ZE661.txt
o4YBAGAPtsyAeRITABlCm7lv5ZE661.txt successful
a audio successful
oYYBAGAPtsuAdlf1ABj4W8LbwdQ894.txt
oYYBAGAPtsuAdlf1ABj4W8LbwdQ894.txt successful
a audio successful
o4YBAGAPtsuAZvOdABnd3LCWN8Y472.txt
o4YBAGAPtsuAZvOdABnd3LCWN8Y472.txt successful
a audio successful


o4YBAGAPtPeAGmV5ABc2XEtZICg902.txt
o4YBAGAPtPeAGmV5ABc2XEtZICg902.txt successful
a audio successful
ooYBAGAPtPeALuYFABjonMBVwSg608.txt
ooYBAGAPtPeALuYFABjonMBVwSg608.txt successful
a audio successful
ooYBAGAPtTGAb30hABg5HFCsV8c224.txt
ooYBAGAPtTGAb30hABg5HFCsV8c224.txt successful
a audio successful
o4YBAGAPtROAG2HjABkRHJH2PlY345.txt
o4YBAGAPtROAG2HjABkRHJH2PlY345.txt successful
a audio successful
ooYBAGAPtPeAIrTnABeOHHl3RA0435.txt
ooYBAGAPtPeAIrTnABeOHHl3RA0435.txt successful
a audio successful
o4YBAGAPtTGAXomnABlW3PwUSyU385.txt
o4YBAGAPtTGAXomnABlW3PwUSyU385.txt successful
a audio successful
o4YBAGAPtPeAUsgoABf8XAxZQ0U262.txt
o4YBAGAPtPeAUsgoABf8XAxZQ0U262.txt successful
a audio successful
ooYBAGAPtPeADRj7ABc0HLGhRZw023.txt
ooYBAGAPtPeADRj7ABc0HLGhRZw023.txt successful
a audio successful
ooYBAGAPtROAH10zABf6HLSQ7HU057.txt
ooYBAGAPtROAH10zABf6HLSQ7HU057.txt successful
a audio successful
o4YBAGAPtPeASB0yABg_3IMlWsg058.txt
o4YBAGAPtPeASB0yABg_3IMlWsg058.txt successful
a audio successful


ooYBAGAKLxyAf-w7ADaCf8ZSQ2Q631.txt
ooYBAGAKLxyAf-w7ADaCf8ZSQ2Q631.txt successful
a audio successful
o4YBAGAKL2SADn-hADI8D8hnqKA744.txt
o4YBAGAKL2SADn-hADI8D8hnqKA744.txt successful
a audio successful
ooYBAGAKL36AdnKeADTib4Y0U0w988.txt
ooYBAGAKL36AdnKeADTib4Y0U0w988.txt successful
a audio successful
ooYBAGAKL2iAcCWZADG9T4fwdo8905.txt
ooYBAGAKL2iAcCWZADG9T4fwdo8905.txt successful
a audio successful
o4YBAGAKLxyAAdNxACTff6UIXVQ372.txt
o4YBAGAKLxyAAdNxACTff6UIXVQ372.txt successful
a audio successful
o4YBAGAKL2eAAaw_ADFYPzP5DjA961.txt
o4YBAGAKL2eAAaw_ADFYPzP5DjA961.txt successful
a audio successful
o4YBAGAKL36ATDevAEIg5xiYG8o479.txt
o4YBAGAKL36ATDevAEIg5xiYG8o479.txt successful
a audio successful
ooYBAGAKLxyAdIZVADiF78_cOuk067.txt
ooYBAGAKLxyAdIZVADiF78_cOuk067.txt successful
a audio successful
o4YBAGAKL0eARcukADQ83_BGfGQ583.txt
o4YBAGAKL0eARcukADQ83_BGfGQ583.txt successful
a audio successful
oYYBAGAKL36AMkvMADJfHxZj-zw307.txt
oYYBAGAKL36AMkvMADJfHxZj-zw307.txt successful
a audio successful


o4YBAGAJJDSAK_omABf8QLqQ2hE189.txt
o4YBAGAJJDSAK_omABf8QLqQ2hE189.txt successful
a audio successful
oYYBAGAJI8eAdPUpABbrmS7GHV8639.txt
oYYBAGAJI8eAdPUpABbrmS7GHV8639.txt successful
a audio successful
o4YBAGAJJDSAPsQ7ABaEgOfsIAo695.txt
o4YBAGAJJDSAPsQ7ABaEgOfsIAo695.txt successful
a audio successful
oYYBAGAJJDuAKRLcABbRANOAXlY932.txt
oYYBAGAJJDuAKRLcABbRANOAXlY932.txt successful
a audio successful
ooYBAGAJJDSAad49ABaxgOEXDKM567.txt
ooYBAGAJJDSAad49ABaxgOEXDKM567.txt successful
a audio successful
ooYBAGAJJDuAJCLFABYNQLHdaGI324.txt
ooYBAGAJJDuAJCLFABYNQLHdaGI324.txt successful
a audio successful
ooYBAGAJJDSANGkwABcNlzFnJG0672.txt
ooYBAGAJJDSANGkwABcNlzFnJG0672.txt successful
a audio successful
o4YBAGAJJBOAe66vABCDPaWa9sM093.txt
o4YBAGAJJBOAe66vABCDPaWa9sM093.txt successful
a audio successful
o4YBAGAJJBOAPFFXABd1QAesaJc448.txt
o4YBAGAJJBOAPFFXABd1QAesaJc448.txt successful
a audio successful
ooYBAGAJJBOAMrB2ABaEgEHares189.txt
ooYBAGAJJBOAMrB2ABaEgEHares189.txt successful
a audio successful
